## Patch Embedding

将一幅图片转化成一个`patch embedding`。

主要有`naive`的实现版本和`conv2d`的实现版本。

首先是将图像切分成一个个`patch`，与`vit`一样。每个`patch`就像`NLP`中的每个`token`一样。在论文中，`patch`的大小取$4 \times 4$, 每个`patch`中都存在三个通道，因此每个`patch`的像素点都是$4 \times 4 \times 3 = 48$个像素点。这是原始像素点，想要得到`patch embedding`的话，我们还需要将其通过一个线性层映射到长度为$C$的向量上。

<img src="../../images/07-swintransformer.jpg" width="80%">


每一个`block`是不会对像素做改变的。在运用`patch merging`的时候才会使得像素发生改变。

1. 方法一：基于unfold API来模仿卷积思路来实现对图像分块，设置kernel size = patch size。得到格式为[bs, num_patch, patch_size]的向量。将张量与[bs, patch_size, model_dim_C]的权重矩阵相乘，得到[bs, num_patch, model_dim_C]的patch embedding。

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

输入图像大小都是按照卷积操作的图像大小给定的，维度为:`[bs * channel * h * w]`。

In [2]:
def image2emb_naive(image, patch_size, weight):
    patch = F.unfold(image, kernel_size=(patch_size, patch_size),
                    stride=(patch_size, patch_size)).transpose(-1, -2)  # [bs, num_patch, patch_depth]
    patch_embedding = patch @ weight
    return patch_embedding

2. 方法二：就是直接通过卷积操作来实现，省去了与权重相乘的过程，卷积输出的通道数就是编码之后的维度。

In [3]:
def image2emb_conv(image, kernel, stride):
    conv_output = F.conv2d(image, kernel, stride=stride) # [bs, oc, oh, ow]
    bs, oc, oh, ow = conv_output.shape
    patch_embedding = conv_output.reshape((bs, oc, oh * ow)).transpose(-1, -2)
    return patch_embedding

## SwinTransformer Block

`SwinTransformer`中的`swin`是`shift`和`window`的结合。`SwinTransformer Block`主要包括两部分，一个是`Window Multi-head Self-attention`和`Shifted Windows Multi-head Self-attention`。

把`patch`做成在不同的`window`内，在`window`内做`self-attention`。每一个`window`内的`self-attention`复杂度和`window`内的`patch`数目成平方关系。而`window`数目与图片大小成线性关系，因此算法整体的复杂度也是会与图片大小成线性关系。`window`与`window`之间的连接是通过`shift window`来去实现的。


### Window Multi-head Self-attention

&emsp;&emsp;这里就是构建多头的注意力机制，并计算其复杂度。

1. 将输入经过`mlp`映射到$q，k，v$。输入$x$的维度为$L \times C$，其中$L$为序列长度，$C$为特征大小。映射矩阵的维度为$C \times C$，所以这两个矩阵相乘的复杂度为$LC^{2}$。有三个映射，所以其复杂度为$3LC^{2}$。将$q，k，v$拆分乘多头的形式，也就是维度$C$拆分成$\frac{C}{n}$, 但是这里的多头计算互不影响，也就是头与头之间并不会做`attention`的计算，因此可以与`batch size`的那一维度进行统一的看待。
2. 之后我们就需要计算`attention`计算过程的复杂度，首先是$q k^{T}$, 其中$q$的矩阵维度为$L \times C$, $k^{T}$的矩阵维度为$C \times L$, 因此这两个矩阵相乘的复杂度为$L^{2}C$。
3. 之后将计算得到的概率与$v$相乘，也就是$L \times L$的矩阵与$L \times C$的矩阵相乘，同样计算其复杂度为$L^{2}C$。
4. 得到最终的`[bs, L, C]`的数据与`mlp`相乘，也就是$L \times C$的矩阵与$C \times C$的矩阵相乘，复杂度为$LC^{2}$。

&emsp;&emsp;此时，我们可以得出总体的复杂度为:

$$
4LC^{2} + 2L^{2}C
$$

可以看出，重头是在做`attention`部分，复杂度与序列长度成平方关系。

在计算$q k^{T}$时，我们需要考虑掩码，让两两无效的位置之间能量为负无穷，经过`softmax`之后，概率就会变成`0`。掩码是在`shift window MHSA`中会用到，在`window MHSA`中不会用到。

在涉及模型，参数的定义的时候，我们需要写成`class`的形式，并且`module`需要继承自`nn.Module`。

In [4]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, model_dim, num_head):
        super(MultiHeadSelfAttention, self).__init__()
        self.num_head = num_head
        
        self.proj_linear_layer = nn.Linear(model_dim, 3*model_dim)
        self.final_linear_layer = nn.Linear(model_dim, model_dim)
    
    def forward(self, input, additive_mask=None):
        bs, seqlen, model_dim = input.shape
        num_head = self.num_head
        
        head_dim = model_dim // self.num_head
        proj_output = self.proj_linear_layer(input) # [bs, seqlen, 3 * model_dim]
        q, k, v = proj_output.chunk(3, dim=-1)  # [bs, seqlen, model_dim]
        
        q = q.reshape(bs, seqlen, num_head, head_dim).transpose(1, 2)
        q = q.reshape(bs * num_head, seqlen, head_dim)
        
        k = k.reshape(bs, seqlen, num_head, head_dim).transpose(1, 2)
        k = k.reshape(bs * num_head, seqlen, head_dim)
        
        v = v.reshape(bs, seqlen, num_head, head_dim).transpose(1, 2)
        v = v.reshape(bs * num_head, seqlen, head_dim)
        
        if additive_mask is None:
            attn_prob = F.softmax(torch.bmm(q, k.transpose(-2, -1))/ math.sqrt(head_dim), dim=-1)
        else:
            additive_mask = additive_mask.tile((num_head, 1, 1))
            attn_prob = F.softmax(torch.bmm(q, k.transpose(-2, -1))/ math.sqrt(head_dim) + additive_mask, dim=-1)
            
        output = torch.bmm(attn_prob, v)  # [bs * num_head, seqlen, head_dim]
        output = output.reshape(bs, num_head, seqlen, head_dim).transpose(1, 2) # [bs, seqlen, num_head, head_dim]
        output = output.reshape(bs, seqlen, model_dim)
        
        output = self.final_linear_layer(output)
        return attn_prob, output

之后，我们就需要去构建带window的MHSA，并计算其复杂度
1. 之前像素做成的patch进一步划分成一个个更大的window。首先需要将三维的patch embedding转换成图片的形式，也就是将[bs, num_patch, model_dim]转成[bs, height, width]的形式。高度乘以宽度需要是patch的总数。采用unfold这个API将patch划分成window。

2. 划分成window之后，我们就可以在window内部去划分MHSA。window数目可以与bs统一对待，因为window数据之间并不做attention。

假设窗的边长是$W$，那么窗内元素的数目就是$W^{2}$，那么计算窗内的总体复杂度是$4W^{2}C^{2} + 2W^{4}C$。假设patch的总数目是$L$, 那么窗的数目就是$L/W^{2}$。因此W-MHSA的复杂度为$4LC^{2} + 2LW^{2}C$。

复杂度对比：

`MHSA`：$4LC^{2} + 2L^{2}C$。

`W-MHSA`：$4LC^{2} + 2LW^{2}C$。

In [5]:
def window_multi_head_self_attention(patch_embedding, mhsa, window_size=4, num_head=2):
    num_patch_in_window = window_size * window_size
    bs, num_patch, patch_depth = patch_embedding.shape
    image_height = image_width = int(math.sqrt(num_patch))
    
    patch_embedding = patch_embedding.transpose(-1, -2)
    patch = patch_embedding.reshape(bs, patch_depth, image_height, image_width)
    window = F.unfold(patch, kernel_size=(window_size, window_size), 
                      stride=(window_size, window_size)).transpose(-1, -2)
    
    bs, num_window, patch_depth_times_num_patch_in_window = window.shape
    window = window.reshape(bs * num_window, patch_depth, num_patch_in_window).transpose(-2, -1)
    
    attn_prob, output = mhsa(window)  # [bs * num_window, num_patch_in_window, patch_depth]
    
    output = output.reshape(bs, num_window, num_patch_in_window, patch_depth)
    return output

### Shifted Windows Multi-head Self-attention

1. 首先将上一步W-MHSA的结果转换成图片的形式，假设已经做了新的window的划分，这一步叫做shift window。为了保持window数目不变，从而有高效计算，需要将图片的patch往左和往上各自滑动半个窗口大小的步长，保持patch所属window类别不变。之后再将图片patch还原成window的数据格式。

2. 由于cycle shift后，每个window虽然形状不规则，但部分window中存在原本不属于同一个窗口的patch，所以需要生成mask。

3. 如何生成`mask`？构建`mask`时，首先需要去构建一个`shift-window`的`patch`所属的`window`类别矩阵，之后对该矩阵往上，往左滑动半个窗口大小的步长，之后通过`unfold`操作得到`[bs, num_window, num_patch_in_window]`形状的类别矩阵。之后将其扩充维度到`[bs, num_window, num_patch_in_window, 1]`这个维度。将该矩阵与其转置矩阵进行作差，得到同类关系矩阵（为0的位置上的patch属于同类，否则属于不同类）。对同类关系矩阵中非零位置用负无穷去填充，对于0的位置用0去填充，这样就构建好了MHSA所需的mask。此mask的形状为[bs, num_window, num_patch_in_window, num_patch_in_window]。

4. 将window转换成三维的格式，[bs*num_window, num_patch_in_window, patch_depth]。
5. 将三维格式的特征连同mask一起送入HSA中计算得到注意力的输出。
6. 将注意力的输出转化成图片patch的格式，[bs, num_window, num_patch_in_window, patch_depth]。
7. 未了恢复位置，需要将图片的patch往右和往下各自滑动半个窗口大小的步长，至此，SW-MHSA计算完毕。

In [6]:
# 辅助函数，将transformer block的结果转换成图片的格式。
def window2image(msa_output):
    bs, num_window, num_patch_in_window, patch_depth = msa_output.shape
    window_size = int(math.sqrt(num_patch_in_window))
    
    image_height = int(math.sqrt(num_window)) * window_size
    image_width = image_height
    
    msa_output = msa_output.reshape(bs, int(math.sqrt(num_window)),
                                        int(math.sqrt(num_window)),
                                        window_size,
                                        window_size,
                                        patch_depth)
    
    msa_output = msa_output.transpose(2, 3) 
    image = msa_output.reshape(bs, image_height*image_width, patch_depth)
    
    image = image.transpose(-1, -2).reshape(bs, patch_depth, image_height, image_width) # 跟卷积格式一致
    return image

# 辅助函数，高效计算swmsa
def shift_window(w_msa_output, window_size, shift_size, generate_mask=False):
    bs, num_window, num_patch_in_window, patch_depth = w_msa_output.shape
    
    w_msa_output = window2image(w_msa_output)  # [bs, depth, h, w]
    bs, patch_depth, image_height, image_width = w_msa_output.shape
    
    rolled_w_msa_input = torch.roll(w_msa_output, shifts=(shift_size, shift_size), dims=(2, 3))
    
    shift_w_msa_input = rolled_w_msa_input.reshape(bs, patch_depth,
                                                   int(math.sqrt(num_window)),
                                                   window_size,
                                                   int(math.sqrt(num_window)),
                                                   window_size
                                                  )
    
    shift_w_msa_input = shift_w_msa_input.transpose(3, 4)
    shift_w_msa_input = shift_w_msa_input.reshape(bs, patch_depth, num_window*num_patch_in_window)
    shift_w_msa_input = shift_w_msa_input.transpose(-1, -2)
    shift_window = shift_w_msa_input.reshape(bs, num_window, num_patch_in_window, patch_depth)
    
    if generate_mask:
        additive_mask = build_mask_for_shifted_wmsa(bs, image_height, image_width, window_size)
    else:
        additive_mask = None
    
    return shift_window, additive_mask

生成mask矩阵之前，首先需要生成类别矩阵index_matrix。

In [7]:
# 构建shift window multi-head attention mask
def build_mask_for_shifted_wmsa(batch_size, image_height, image_width, window_size):
    index_matrix = torch.zeros(image_height, image_width)
    
    for i in range(image_height):
        for j in range(image_width):
            row_times = (i + window_size//2) // window_size
            col_times = (j + window_size//2) // window_size
            index_matrix[i, j] = row_times * (image_height // window_size) + col_times + 1
    
    rolled_index_matrix = torch.roll(index_matrix, shifts=(-window_size//2, -window_size//2), dims=(0, 1))
    rolled_index_matrix = rolled_index_matrix.unsqueeze(0).unsqueeze(0)  # [bs, ch, h, w]
    
    c = F.unfold(rolled_index_matrix, kernel_size=(window_size, window_size),
                 stride=(window_size, window_size)).transpose(-1, -2)
    
    c = c.tile(batch_size, 1, 1) # [bs, num_window, num_patch_in_window]
    
    bs, num_window, num_patch_in_window = c.shape
    c1 = c.unsqueeze(-1) # [bs, num_window, num_patch_in_window, 1]
    c2 = (c1 - c1.transpose(-1, -2)) == 0 # [bs, num_window, num_patch_in_window, num_patch_in_window]
    valid_mask = c2.to(torch.float32)
    
    additive_mask = (1 - valid_mask)*(-1e-9) # [bs, num_window, num_patch_in_window, num_patch_in_window]
    
    additive_mask = additive_mask.reshape(bs*num_window, num_patch_in_window, num_patch_in_window)
    
    return additive_mask

In [8]:
# 主函数
def shift_window_multi_head_self_attention(w_msa_output, mhsa, window_size=4, num_head=2):
    """
    w_msa_output: 上一步shift window的输出。
    mhsa: 新的实例化的mhsa的对象。
    """
    bs, num_window, num_patch_in_window, patch_depth = w_msa_output.shape
    
    shifted_w_msa_input, additive_mask = shift_window(w_msa_output, window_size, 
                                                      shift_size=-window_size//2, generate_mask=True)
    shifted_w_msa_input = shifted_w_msa_input.reshape(bs*num_window, num_patch_in_window, patch_depth)
    
    attn_prob, output = mhsa(shifted_w_msa_input, additive_mask=additive_mask)
    output = output.reshape(bs, num_window, num_patch_in_window, patch_depth)
    
    output, _ = shift_window(output, window_size, shift_size=window_size//2, generate_mask=False)
    return output

## Patch Merging

`Patch Merging`主要实现的是像素的降低，将周围的`patch`浓缩成一个`patch`。

`Patch Merging`是将`window`格式的特征转换成图片`patch`格式。利用`unfold`操作，按照`merge_size * merge_size`的大小得到新的`patch`，形状为`[bs, num_patch_new, merge_size * merge_size * patch_depth_old]`。

使用全连接层对`depth`进行`0.5`倍的降维，输出`patch embedding`的形状格式`[bs, num_patch, patch_depth]`。

举例说明：以`merge_size = 2`为例，经过`PatchMerging`后，数目减少为之前的`1/4`，但是`depth`增大为原来的两倍，而不是`4`倍。

In [9]:
class PatchMerging(nn.Module):
    def __init__(self, model_dim, merge_size, output_depth_scale=0.5):
        super(PatchMerging, self).__init__()
        self.merge_size = merge_size
        self.proj_layer = nn.Linear(
            model_dim * merge_size * merge_size,
            int(model_dim * merge_size * merge_size * output_depth_scale)
        )
    
    def forward(self, input):
        bs, num_window, num_patch_in_window, patch_depth = input.shape
        
        window_size = int(math.sqrt(num_patch_in_window))
        
        input = window2image(input) # [bs, patch_depth, image_h, image_w]
        
        merge_window = F.unfold(input, kernel_size=(self.merge_size, self.merge_size),
                               stride=(self.merge_size, self.merge_size)).transpose(-1, -2)
        
        merge_window = self.proj_layer(merge_window) # [bs, num_patch, new_patch_depth]
        
        return merge_window

## 如何构建swin transformer block

每个block包含LayerNorm，W-MHSA，MLP，SW-MHSA、残差连接等模块。
输入是patch embedding格式
每个MLP包含两层，分别是4*model_dim和model_dim大小。
输出的是window的数据格式，[bs, num_window, num_patch_in_window, patch_depth]。
需要注意残差连接对数据形状的要求。

In [10]:
class SwinTransformerBlock(nn.Module):
    def __init__(self, model_dim, window_size, num_head):
        super(SwinTransformerBlock, self).__init__()
        self.layer_norm1 = nn.LayerNorm(model_dim)
        self.layer_norm2 = nn.LayerNorm(model_dim)
        self.layer_norm3 = nn.LayerNorm(model_dim)
        self.layer_norm4 = nn.LayerNorm(model_dim)
        
        self.wsma_mlp1 = nn.Linear(model_dim, 4*model_dim)
        self.wsma_mlp2 = nn.Linear(4*model_dim, model_dim)
        
        self.swsma_mlp1 = nn.Linear(model_dim, 4*model_dim)
        self.swsma_mlp2 = nn.Linear(4*model_dim, model_dim)
        
        self.mhsa1 = MultiHeadSelfAttention(model_dim, num_head)
        self.mhsa2 = MultiHeadSelfAttention(model_dim, num_head)
    
    def forward(self, input):
        bs, num_patch, patch_depth = input.shape
        input1 = self.layer_norm1(input)
        w_msa_output = window_multi_head_self_attention(input, self.mhsa1, window_size=4, num_head=2)
        bs, num_window, num_patch_in_window, patch_depth = w_msa_output.shape
        w_msa_output = input + w_msa_output.reshape(bs, num_patch, patch_depth)
        output1 = self.wsma_mlp2(self.wsma_mlp1(self.layer_norm2(w_msa_output)))
        
        input2 = self.layer_norm3(output1)
        input2 = input2.reshape(bs, num_window, num_patch_in_window, patch_depth)
        sw_msa_output = shift_window_multi_head_self_attention(input2, self.mhsa2, window_size=4, num_head=2)
        sw_msa_output = output1 + sw_msa_output.reshape(bs, num_patch, patch_depth)
        output2 = self.swsma_mlp2(self.swsma_mlp1(self.layer_norm4(sw_msa_output)))
        output2 += sw_msa_output
        
        output2 = output2.reshape(bs, num_window, num_patch_in_window, patch_depth)
        
        return output2

## 如何构建swin transformermodel

输入是图片，首先对图片进行分块得到patch embedding。

经过第一个stage，进行patch merging，再进行第二个stage。以此类推

对最后一个block的输出转换成patch embedding的格式，[bs, num_patch, patch_depth]。

对patch embeding在时间维度进行平均池化，并映射到分类层，得到分类的logits。


In [11]:
class SwinTransformerModel(nn.Module):
    def __init__(self, input_image_channel=3, patch_size=4, model_dim_C=8, num_class=10,
                window_size = 4, num_head = 2, merge_size = 2):
        super(SwinTransformerModel, self).__init__()
        patch_depth = patch_size * patch_size * input_image_channel
        self.patch_size = patch_size
        self.model_dim_C = model_dim_C
        self.num_class = num_class
        
        self.patch_embedding_weight = nn.Parameter(torch.randn(patch_depth, model_dim_C))
        
        self.block1 = SwinTransformerBlock(model_dim_C, window_size, num_head)
        self.block2 = SwinTransformerBlock(model_dim_C*2, window_size, num_head)
        self.block3 = SwinTransformerBlock(model_dim_C*4, window_size, num_head)
        self.block4 = SwinTransformerBlock(model_dim_C*8, window_size, num_head)
        
        self.patch_merging1 = PatchMerging(model_dim_C, merge_size)
        self.patch_merging2 = PatchMerging(model_dim_C*2, merge_size)
        self.patch_merging3 = PatchMerging(model_dim_C*4, merge_size)
        
        self.final_layer = nn.Linear(model_dim_C*8, num_classes)
        
    def forward(self, image):
        patch_embedding_naive = image2emb_naive(image, self.patch_size, self.patch_embedding_weight)
        
        # block1
        patch_embedding = patch_embedding_naive
        print(patch_embedding.shape)
        
        sw_msa_output = self.block1(patch_embedding)
        print("block 1 output", sw_msa_output.shape)
        
        merged_patch1 = self.patch_merging1(sw_msa_output)
        sw_msa_output_1 = self.block2(merged_patch1)
        print("block 2 output", sw_msa_output_1.shape)
        
        merged_patch2 = self.patch_merging2(sw_msa_output_1)
        sw_msa_output_2 = self.block3(merged_patch2)
        print("block 3 output", sw_msa_output_2.shape)
        
        merged_patch3 = self.patch_merging3(sw_msa_output_2)
        sw_msa_output_3 = self.block4(merged_patch3)
        print("block 4 output", sw_msa_output_3.shape)
        
        bs, num_window, num_patch_in_window, patch_depth = sw_msa_output_3.shape
        sw_msa_output_3 = sw_msa_output_3.reshape(bs, -1, patch_depth)
        
        pool_output = torch.mean(sw_msa_output_3, dim=1)
        logits = self.final_layer(pool_output)
        print("logits", logits.shape)
        return logits

## 模型测试



In [12]:
if __name__ == "__main__":
    bs, ic, image_h, image_w = 4, 3, 256, 256
    patch_size = 4
    model_dim_C = 8  # 一开始patch embedding的大小
    max_num_token = 16
    num_classes = 10
    window_size = 4
    num_head = 2
    merge_size = 2
    
    patch_depth = patch_size * patch_size * ic
    image = torch.randn(bs, ic, image_h, image_w)
    model = SwinTransformerModel(ic, patch_size, model_dim_C, num_classes, window_size, num_head, merge_size)
    
    logits = model(image)
    print(logits)

torch.Size([4, 4096, 8])
block 1 output torch.Size([4, 256, 16, 8])
block 2 output torch.Size([4, 64, 16, 16])
block 3 output torch.Size([4, 16, 16, 32])
block 4 output torch.Size([4, 4, 16, 64])
logits torch.Size([4, 10])
tensor([[ 0.1698, -0.9871, -0.3158, -0.0033,  0.7429, -0.2816, -0.0267, -0.0669,
         -0.1649, -0.4092],
        [ 0.1386, -0.9790, -0.3111, -0.0200,  0.7560, -0.2866, -0.0131, -0.0499,
         -0.1631, -0.4140],
        [ 0.1590, -0.9869, -0.3212, -0.0055,  0.7552, -0.2835, -0.0131, -0.0642,
         -0.1543, -0.4135],
        [ 0.1698, -0.9873, -0.3231, -0.0243,  0.7357, -0.2724, -0.0145, -0.0723,
         -0.1595, -0.4056]], grad_fn=<AddmmBackward0>)
